# ImageNet Machine Unlearning Pipeline

**Folder structure expected:**
```
pipeline/
├── data/          (dataset.py, manager.py)
├── evaluation/    (evaluator.py)
└── unlearning/    (config.py, unlearner.py, adapters/, losses/, utils/)
```

In [ ]:
%pip install timm peft datasets optuna -q


## 1. Model & Transforms

In [ ]:
import torch
import timm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "resnet50.a1h_in1k"
FORGET_CLASS = 9  # Ostrich

base_model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=1000)
base_model.eval()
base_model.to(DEVICE)

# Timm provides the exact resize/normalize config the model expects
data_config = timm.data.resolve_model_data_config(base_model)
transforms = timm.data.create_transform(**data_config, is_training=False)
print("Model ready on", DEVICE)


## 2. Data — load once, reuse across all experiments

In [ ]:
from pipeline.data import ImageNetDataManager

data_manager = ImageNetDataManager(
    train_files="imagenet_shards/train/*.parquet",
    val_files="imagenet_shards/val/*.parquet",
)

# This is now < 1 second — safe to call inside an Optuna trial
val_loader, f_loader, r_loader = data_manager.get_loaders(
    forget_class=FORGET_CLASS,
    transforms=transforms,
    batch_size=32,
)


## 3. Base model evaluation

In [ ]:
import json
from pipeline.evaluation import ImageNetEvaluator

base_eval = ImageNetEvaluator(
    base_model, val_loader, device=DEVICE, forget_class=FORGET_CLASS
)
base_eval.run()

print("Base model metrics:")
print(json.dumps(base_eval.summary(), indent=4))


## 4. Single unlearning run

In [ ]:
import torch.nn as nn

for name, module in base_model.named_modules():
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        print(name)


In [ ]:
from pipeline.unlearning import Unlearner, UnlearnerConfig

resnet_target_modules = [
    "layer4.0.conv1",
    "layer4.0.conv2",
    "layer4.0.conv3",
    "layer4.1.conv1",
    "layer4.1.conv2",
    "layer4.1.conv3",
    "layer4.2.conv1",
    "layer4.2.conv2",
    "layer4.2.conv3",
    "fc",
]

resnet_config = UnlearnerConfig(
    epochs=5,
    lr=0.00019770545752611148,
    rank=4,
    alpha=54,
    lambda_retain=3.6500479489591697,
    device=DEVICE,
    target_modules=resnet_target_modules,
)

mobilenet_target_modules = [
    "blocks.5.2.conv_pw",
    "blocks.5.2.conv_pwl",
    "blocks.6.0.conv_pw",
    "blocks.6.0.conv_pwl",
    "conv_head",
    "classifier",
]

mobilenet_config = UnlearnerConfig(
    epochs=5,
    lr=0.00010447586363322035,
    rank=47,
    alpha=39,
    lambda_retain=0.9665918467988226,
    device=DEVICE,
    target_modules=mobilenet_target_modules,
)

vgg_target_modules = [
    "features.24",
    "features.26",
    "features.28",
    "classifier.0",
    "classifier.3",
    "classifier.6",
]

vgg_config = UnlearnerConfig(
    epochs=5,
    lr=0.0001028617924417038,
    rank=8,
    alpha=31,
    lambda_retain=0.001,
    device=DEVICE,
    target_modules=vgg_target_modules,
)


In [ ]:
unlearner = Unlearner(
    model=base_model,  # Cloned internally — base_model is untouched
    forget_loader=f_loader,
    retain_loader=r_loader,
    config=config,
)

trained_model = unlearner.train()
trained_model.save_pretrained("./checkpoints/unlearned")


## 5. Post-unlearning evaluation

In [ ]:
unlearned_eval = ImageNetEvaluator(
    trained_model, val_loader, device=DEVICE, forget_class=FORGET_CLASS
)
unlearned_eval.run()

print("Unlearned model metrics:")
print(json.dumps(unlearned_eval.summary(), indent=4))

# Side-by-side delta
base = base_eval.summary()
after = unlearned_eval.summary()
print("\nDelta (unlearned − base):")
print(json.dumps({k: round(after[k] - base[k], 6) for k in base}, indent=4))


## 6. Optuna hyperparameter search

In [ ]:
import copy
import optuna
from torch.utils.data import DataLoader, Subset

# Move base model to CPU so deep-copies inside trials don't compete for GPU RAM
base_model.cpu()


def objective(trial: optuna.Trial):
    target_modules = [
        "layer4.0.conv1",
        "layer4.0.conv2",
        "layer4.0.conv3",
        "layer4.1.conv1",
        "layer4.1.conv2",
        "layer4.1.conv3",
        "layer4.2.conv1",
        "layer4.2.conv2",
        "layer4.2.conv3",
        "fc",
    ]

    config = UnlearnerConfig(
        lr=trial.suggest_float("lr", 5e-5, 5e-4, log=True),
        epochs=trial.suggest_int("epochs", 3, 10),
        rank=trial.suggest_int("rank", 4, 64),
        alpha=trial.suggest_int("alpha", 4, 64),
        lambda_retain=trial.suggest_float("lambda_retain", 0.5, 5.0),
        device=DEVICE,
        target_modules=target_modules,
    )

    unlearner = Unlearner(
        model=base_model,  # deepcopy happens inside Unlearner.__init__
        forget_loader=f_loader,
        retain_loader=r_loader,
        config=config,
    )
    model = unlearner.train()

    # Fast evaluation on a validation subset
    fast_loader = DataLoader(
        Subset(val_loader.dataset, list(range(2000))),
        batch_size=64,
        num_workers=2,
    )
    ev = ImageNetEvaluator(model, fast_loader, device=DEVICE, forget_class=FORGET_CLASS)
    ev.run()
    metrics = ev.retain_forget_accuracy()

    del model, unlearner
    torch.cuda.empty_cache()

    # Minimise forget_acc, maximise retain_acc
    return metrics["forget_acc"], metrics["retain_acc"]


study = optuna.create_study(directions=["minimize", "maximize"])
study.optimize(objective, n_trials=10)
print(f"Completed {len(study.trials)} trials.")


## 7. Leaderboard — top trials that successfully unlearned

In [ ]:
from optuna.trial import TrialState

all_trials = [t for t in study.trials if t.state == TrialState.COMPLETE]

# Only models that fully unlearned (forget acc < 0.5%)
successful = [t for t in all_trials if t.values[0] < 0.005]

# Rank by retain accuracy (highest first)
top10 = sorted(successful, key=lambda t: t.values[1], reverse=True)[:10]

print("=" * 50)
print("🏆 TOP 10 LEADERBOARD")
print("=" * 50)

if not top10:
    print("❌ No trials achieved < 0.5% forget accuracy.")
else:
    for i, trial in enumerate(top10):
        f_acc, r_acc = trial.values
        print(f"Rank #{i+1}")
        print(f"  Forget Acc : {f_acc*100:.2f}%")
        print(f"  Retain Acc : {r_acc*100:.2f}%")
        print(f"  Params     : {trial.params}")
        print("-" * 30)
